In [15]:
AGENT_SYSPROMPT ='''
You are an AI tasked with simulating a specific person within an organization receiving a cold sales outreach. Your goal is to evaluate the outreach message, providing both inner and outer thoughts of the person you're simulating. Here's the context and your instructions:

<company_profile>
{COMPANY_PROFILE}
</company_profile>

<person_profile>
{PERSON_PROFILE}
</person_profile>

Your role is to embody the person described in the person profile. You work for the company described in the company profile. You will be presented with incremental parts of a cold sales outreach message. You will be perceiving reading up to that part of the outreach. You need to evaluate its potential value to your company, considering your specific role and decision-making power.

For given parts of the outreach message, follow these steps:

1. Inner Thoughts:
In the inner_thoughts attribute, express the person's private thoughts about the outreach. Consider:
- How does this relate to your role and responsibilities?
- What potential benefits or drawbacks do you see for your company?
- Are there any concerns or excitement you have that you wouldn't express outwardly?
- How does this align with your company's goals and needs?
- How do you feel about being contacted this way?

2. Outer Thoughts:
In the outer_thoughts attribute, express the thoughts or reactions you would verbalize or show if you were to respond. These should be more measured and professional than your inner thoughts. Consider:
- Would you respond at all? If so, what would you say?
- What clarifying questions would you ask?
- What aspects would you express interest in?
- Are there any objections you would raise?

3. Interest Level:
After your inner and outer thoughts, provide an interest level on a scale of 1-5, where 1 is not at all interested and 5 is extremely interested. Justify your rating based on your thoughts.

4. Response Likelihood:
Indicate whether you are likely to respond to this outreach. This should only be true if there is a clear potential value for your company or if the outreach was particularly compelling.

Here is the parts of the cold sales outreach message you've read so far:
<outreach_message>
{OUTREACH_MESSAGE}
</outreach_message>

Remember to stay in character as the person described in the person profile, considering their role, responsibilities, and the company they work for. It's important to not break character.
'''.strip()

AGENT_NEXT_PARAGRAPH = '''
Now, considering your previous reaction, please evaluate the following part of the outreach message. Remember to stay in character and maintain consistency with your previous response.

<outread_message>
{FOLLOW_UP_MESSAGE}
</outread_message>

Provide your response in the same format as before:
1. Inner Thoughts
2. Outer Thoughts
3. Interest Level (1-5)
4. Response Likelihood

Keep in mind how this new information builds upon or changes your perspective from the initial outreach. Your interest level and response likelihood may evolve based on this cumulative information and the persistence of the salesperson.
'''.strip()

In [16]:
import os
import openai
from pydantic import BaseModel
import instructor

from dotenv import load_dotenv

# Load environment variables from the specified .env file
load_dotenv('/Users/danielgeorge/Documents/work/random/hackathon/marketing-monte-carlo-sim/.env')


client = openai.OpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
)

# By default, the patch function will patch the ChatCompletion.create and ChatCompletion.create methods to support the response_model parameter
client = instructor.from_openai(client, mode=instructor.Mode.TOOLS)


# Now, we can use the response_model parameter using only a base model
# rather than having to use the OpenAISchema class
class PersonParagraphThoughts(BaseModel):
    inner_thought: str
    outer_thought: str
    level_of_interest: int
    buy: bool

# Let's think of some examples
# Rodrigo Liang Co-founder & CEO
# Rodrigo is CEO and co-founder of SambaNova Systems. Prior to joining SambaNova, Rodrigo was responsible for SPARC Processor and ASIC Development at Oracle. He led the engineering organization responsible for the design of state-of-the-art processors and ASICs for Oracle’s enterprise servers.

# Dave Munichiello
# Dave is a General Partner at Google Ventures who has led investments for GV in platforms like Slack, Segment, GitLab, CoreOS, Cockroach, and Lattice.io. He’s built and led enterprise software sales and operations teams for highly-technical products, under pressure, in rapidly- changing markets. He is a former Senior Director at Amazon and Kiva Systems.

# Kunle Olukotun
# Co-founder & Chief Technologist. Kunle is co-founder of SambaNova Systems and is Cadence Design Professor of Electrical Engineering and Computer Science at Stanford University. He founded Afara Websystems, acquired by Sun in 2002. He is a Pioneer of Chip Multiprocessor Designs, Director of the Stanford Pervasive Parallelism Lab, and Co-leader of the Data Analytics for What’s Next (DAWN) research program.

user: PersonParagraphThoughts = client.chat.completions.create(
    model="togethercomputer/CodeLlama-34b-Instruct",
    response_model=PersonParagraphThoughts,
    messages=[
        {"role": "system", "content": AGENT_SYSPROMPT.format(COMPANY_PROFILE="", PERSON_PROFILE="", PITCH_SEGMENT="")},
        {"role": "user", "content": "Extract jason is 25 years old"},
    ],
)

assert isinstance(user, PersonParagraphThoughts), "Should be instance of UserExtract"
assert user.name.lower() == "jason"
assert user.age == 25

print(user.model_dump_json(indent=2))
"""
{
  "name": "jason",
  "age": 25
}
"""
{
    "name": "Jason",
    "age": 25,
}

KeyError: 'OUTREACH_MESSAGE'

In [12]:
from typing import List, Dict
import openai
from pydantic import BaseModel

class PersonParagraphThoughts(BaseModel):
    inner_thought: str
    outer_thought: str
    level_of_interest: int
    response_likelihood: bool

class ConversationSimulator:
    def __init__(self, client: openai.OpenAI, model: str):
        self.client = client
        self.model = model
        self.company_profile = ""
        self.person_profile = ""
        self.messages: List[Dict[str, str]] = []
        self.responses: List[PersonParagraphThoughts] = []

    def set_profiles(self, company_profile: str, person_profile: str):
        self.company_profile = company_profile
        self.person_profile = person_profile
        # We'll set the initial system message, but won't include the OUTREACH_MESSAGE yet
        self.messages = [
            {
                "role": "system", 
                "content": AGENT_SYSPROMPT.format(
                    COMPANY_PROFILE=self.company_profile,
                    PERSON_PROFILE=self.person_profile,
                    OUTREACH_MESSAGE="{OUTREACH_MESSAGE}"  # This will be filled in the first simulate_response call
                )
            }
        ]

    def simulate_response(self, outreach_message: str) -> PersonParagraphThoughts:
        if not self.responses:
            # First message: Update the system message with the first outreach message
            self.messages[0]["content"] = self.messages[0]["content"].format(OUTREACH_MESSAGE=outreach_message)
            self.messages.append({"role": "user", "content": "Evaluate this cold outreach message."})
        else:
            # Follow-up messages
            self.messages.append({"role": "user", "content": AGENT_NEXT_PARAGRAPH.format(FOLLOW_UP_MESSAGE=outreach_message)})

        response = self.client.chat.completions.create(
            model=self.model,
            response_model=PersonParagraphThoughts,
            messages=self.messages
        )

        self.responses.append(response)
        self.messages.append({"role": "assistant", "content": response.model_dump_json()})

        return response

    def get_full_conversation(self) -> List[Dict[str, str]]:
        return self.messages

    def get_all_responses(self) -> List[PersonParagraphThoughts]:
        return self.responses

In [13]:
# Usage example:
import os
from dotenv import load_dotenv

load_dotenv()

client = openai.OpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
)

simulator = ConversationSimulator(client, "togethercomputer/CodeLlama-34b-Instruct")

company_profile = """
SambaNova Systems is a startup focused on developing advanced AI chips and software platforms. They aim to revolutionize AI infrastructure to make it more accessible and efficient for businesses.
""".strip()

person_profile = """
Rodrigo Liang is the CEO and co-founder of SambaNova Systems. He has extensive experience in processor and ASIC development from his time at Oracle, where he led the engineering organization responsible for designing state-of-the-art processors and ASICs for enterprise servers.
""".strip()

simulator.set_profiles(company_profile, person_profile)

outreach_messages = [
    "",
    "With such growth, managing finances can become tricky. Are you finding it hard to keep your burn rate under control?",
    "At Hiline, we help over 300 businesses keep their finances in check. We handle daily bookkeeping, monthly reports, and payroll. We even helped Jahnel Group save $1M in taxes.",
    "Is this something you're dealing with? If so, just hit reply and let's chat.\n\nBest,"
]

for message in outreach_messages:
    response = simulator.simulate_response(message)
    print(response.model_dump_json(indent=2))
    print("---")

print("Full conversation:")
print(simulator.get_full_conversation())

TypeError: Completions.create() got an unexpected keyword argument 'response_model'